## Generalised Recommender based on popular movies by 'Genre'

#### Author: C.K. Jagannath

In [33]:
# Importing the required modules
import pandas as pd
import numpy as np
from pandas import DataFrame as df

In [34]:
# Reads movie information(movieId, imdbId, tmdbId)
movie_imdbid_tmdbid = pd.read_csv('ml-25m/links.csv')

In [35]:
# Reads rating data(contains userId, movieId, rating)
movie_ratings = pd.read_csv('ml-25m/ratings.csv')

In [36]:
# This step is done to count no. of ratings for each movie
# To do this, we first group the 'movie_ratings' dataframe based on 'movieId' column and turn it into a frame with column 'votes'
df_votes = movie_ratings.groupby('movieId')['movieId'].count().to_frame(name='votes').reset_index()

In [37]:
# This step is done to get the average rating for each movie
# To do this, we first group the 'movie_ratings' df based on 'movieId' column and obtain 'rating' column to perform mean and turn it into a frame 
df_avgrating = movie_ratings.groupby('movieId')['rating'].mean().apply(lambda x: round(x,3)).to_frame(name='avg_rating').reset_index()

In [38]:
# Here we merge the 'votes' df and 'avg_rating' df based on 'movieId'
df_votes_avgrating_merged = df.merge(df_votes,df_avgrating,on='movieId')

In [39]:
# Here we merge the obtained df with 'votes' and 'avg_rating' to the df which contains movie information i.e. 'ids'
df_merged_moviedata = df.merge(movie_imdbid_tmdbid,df_votes_avgrating_merged,how='left',on='movieId')

In [40]:
# Reading movie-titles & genres data
movie_name_genres = pd.read_csv('ml-25m/movies.csv')

In [41]:
# Merging titles & genres with 'ids' df based on 'movieId'
merged_names_ids = pd.merge(movie_name_genres,movie_imdbid_tmdbid,on='movieId')

In [42]:
# This gives us the required df to work on, which contains whole information including votes and avg_rating
merged_moviedata = md = df.merge(merged_names_ids,df_merged_moviedata,on=['movieId','tmdbId','imdbId'])

In [43]:
# Modifying the 'genres' column in such a way that 'no genres listed' will be [] and others will be converted to list of genres
md['genres'] = md['genres'].apply(lambda x: x.split('|') if x != '(no genres listed)' else [])

In [44]:
vote_counts = md[md['votes'].notnull()]['votes'].astype(int) # getting only the rows of the df with non-null in 'votes' column(i.e only movies with atleast a vote)
vote_averages = md[md['avg_rating'].notnull()]['avg_rating'] # getting only the rows of the df with non-null in 'avg_rating' column(i.e only movies with atleast a vote)
C = round(vote_averages.mean(), 1)    
m = int(vote_counts.quantile(0.95))
#C is mean votes across whole report and m is min votes required to be qualified for building top movies chart i.e movie should be in 95th percentile

In [45]:
C, m

(3.1, 1503)

In [46]:
# getting movies with min. votes greater than C
qualified = md[(md['votes'] >= m) & (md['votes'].notnull()) & (md['avg_rating'].notnull())][['movieId', 'title', 'genres', 'imdbId', 'tmdbId', 'votes', 'avg_rating']]

In [47]:
# Therefore 2953 movies qualify to be on our top movies chart

In [48]:
def weighted_rating(x):
    v = x['votes']
    R = x['avg_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)  # IMDB's weighted rating formula
# C, m are same as earlier defined and v is votes for a particular movie and R is its corresponding average rating 

In [49]:
#getting weighted rating for each movie
qualified['wei_rating'] = qualified.apply(weighted_rating, axis=1)
qualified['avg_rating'] = qualified['avg_rating'].apply(lambda x: round(x, 1))

In [50]:
# getting the top 250 movies with highest weighted rating in descending order
qualified = qualified.sort_values('wei_rating', ascending=False).head(250)

In [51]:
qualified.head(15)

,movieId,title,genres,imdbId,tmdbId,votes,avg_rating,wei_rating
314,318,"Shawshank Redemption, The (1994)","[Crime, Drama]",111161,278.0,81482.0,4.4,4.390201
840,858,"Godfather, The (1972)","[Crime, Drama]",68646,238.0,52498.0,4.3,4.289933
49,50,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]",114814,629.0,55366.0,4.3,4.252708
522,527,Schindler's List (1993),"[Drama, War]",108052,424.0,60411.0,4.2,4.220132
1190,1221,"Godfather: Part II, The (1974)","[Crime, Drama]",71562,240.0,34188.0,4.3,4.213066
2867,2959,Fight Club (1999),"[Action, Crime, Drama, Thriller]",137523,550.0,58773.0,4.2,4.199873
1164,1193,One Flew Over the Cuckoo's Nest (1975),[Drama],73486,510.0,36058.0,4.2,4.174223
292,296,Pulp Fiction (1994),"[Comedy, Crime, Drama, Thriller]",110912,680.0,79672.0,4.2,4.168837
883,904,Rear Window (1954),"[Mystery, Thriller]",47396,567.0,20162.0,4.2,4.159052
734,750,Dr. Strangelove or: How I Learned to Stop Worr...,"[Comedy, War]",57012,935.0,26714.0,4.2,4.156555


In [52]:
md['genres']

0        [Adventure, Animation, Children, Comedy, Fantasy]
1                           [Adventure, Children, Fantasy]
2                                        [Comedy, Romance]
3                                 [Comedy, Drama, Romance]
4                                                 [Comedy]
                               ...                        
62418                                              [Drama]
62419                                        [Documentary]
62420                                      [Comedy, Drama]
62421                                                   []
62422                           [Action, Adventure, Drama]
Name: genres, Length: 62423, dtype: object

#### This first takes each row from 'genres' column, converts them into a series, then it is stacked so that the corresponding columns are removed and added as an extra level as index. Then we remove that unnecessary extra level using reset_index on level 1

In [53]:
s = md.apply(lambda x: pd.Series(x['genres'], dtype=object),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

In [54]:
s.head(10)

0    Adventure
0    Animation
0     Children
0       Comedy
0      Fantasy
1    Adventure
1     Children
1      Fantasy
2       Comedy
2      Romance
Name: genre, dtype: object

In [55]:
# we add the 's' df to our df and removing 'genres' column
gen_md = md.drop('genres', axis=1).join(s)
gen_md.dropna(inplace = True)

In [56]:
gen_md

,movieId,title,imdbId,tmdbId,votes,avg_rating,genre
0,1,Toy Story (1995),114709,862.0,57309.0,3.894,Adventure
0,1,Toy Story (1995),114709,862.0,57309.0,3.894,Animation
0,1,Toy Story (1995),114709,862.0,57309.0,3.894,Children
0,1,Toy Story (1995),114709,862.0,57309.0,3.894,Comedy
0,1,Toy Story (1995),114709,862.0,57309.0,3.894,Fantasy
...,...,...,...,...,...,...,...
62420,209163,Bad Poems (2018),6755366,553036.0,1.0,4.500,Comedy
62420,209163,Bad Poems (2018),6755366,553036.0,1.0,4.500,Drama
62422,209171,Women of Devil's Island (1962),55323,79513.0,1.0,3.000,Action
62422,209171,Women of Devil's Island (1962),55323,79513.0,1.0,3.000,Adventure


### This is a function for filtering movie chart based on genre specified

In [57]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['votes'].notnull()]['votes'].astype('int')
    vote_averages = df[df['avg_rating'].notnull()]['avg_rating']
    C = round(vote_averages.mean(), 1)
    m = int(vote_counts.quantile(percentile))
    
    qualified = df[(df['votes'] >= m) & (df['votes'].notnull()) & (df['avg_rating'].notnull())][['movieId', 'title', 'imdbId', 'tmdbId', 'votes', 'avg_rating']]
    qualified['wei_rating'] = qualified.apply(weighted_rating, axis=1)
    qualified['avg_rating'] = qualified['avg_rating'].apply(lambda x: round(x, 1))
    
    qualified['wei_rating'] = qualified.apply(lambda x: (x['votes']/(x['votes']+m) * x['avg_rating']) + (m/(m+x['votes']) * C), axis=1)
    qualified = qualified.sort_values('wei_rating', ascending=False).head(250)
    return qualified

In [58]:
build_chart('Children').head(15)

,movieId,title,imdbId,tmdbId,votes,avg_rating,wei_rating
5859,5971,My Neighbor Totoro (Tonari no Totoro) (1988),96283,8392.0,9340.0,4.2,4.144842
1120,1148,Wallace & Gromit: The Wrong Trousers (1993),108598,531.0,15270.0,4.1,4.068511
729,745,Wallace & Gromit: A Close Shave (1995),112691,532.0,12241.0,4.1,4.060996
6238,6350,Laputa: Castle in the Sky (Tenkû no shiro Rapy...,92067,10515.0,5448.0,4.1,4.016073
12429,60069,WALL·E (2008),910970,10681.0,27374.0,4.0,3.983827
13362,68954,Up (2009),1049413,14160.0,25127.0,4.0,3.982406
932,953,It's a Wonderful Life (1946),38650,1585.0,15384.0,4.0,3.971580
2712,2804,"Christmas Story, A (1983)",85334,850.0,12980.0,4.0,3.966493
1192,1223,"Grand Day Out with Wallace and Gromit, A (1989)",104361,530.0,7695.0,4.0,3.944751
8921,26662,Kiki's Delivery Service (Majo no takkyûbin) (1...,97814,16859.0,4510.0,4.0,3.909274
